# GeoplotLib

- Mobility trace for the city of São Paulo
- URL: http://interscity.org/open_data/
- ZIP FILE: https://www.dropbox.com/s/hatzbujfspte12i/car-bus-simulation.zip?dl=0#

In [92]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import geoplotlib

# CSV DATASET from InterSCimulator
TRACE_DATA = '/home/tallys/git/master-research/src/datasets/car-bus-simulation/output/events_lat_long.csv'
COLUMNS = ['time', 'action', 'vid', 'lat', 'lon']

# Define input size number of rows to be read from the csv into the pandas dataframe
UNITY=1
VID='4858_52'#'6311_1'
SMALL=1000000
MEDIUM=5000000
LARGE=None
INPUT_SIZE=SMALL

mdf = pd.read_csv(TRACE_DATA, nrows=INPUT_SIZE, names=COLUMNS, delimiter=";", header=None)
       
# All data points from a given vehicle id
def filter_by_id(dataframe, vid):
    return dataframe.loc[dataframe['vid'] == vid]

# The first time a vehicle id registered an action in the simulation
def origin(dataframe,id):
    return filter_by_id(dataframe, id).iloc[[0]]
        
# The last time a vehicle id registered an action in the simulation
def destiny(dataframe, id):
    return filter_by_id(dataframe, id).iloc[[-1]]

tool_tip = lambda x: 'time: ' + str(x["time"]) + ', lat: ' + str(x["lat"]) + ', lon: ' + str(x["lon"])


## a) Plot trajectory points of vehicle

- Select vehicle id (vid): 4858_52
- use geoplotlib.dot function to plot the data

In [98]:
import geoplotlib
VID='2280_1'
points_df = filter_by_id(mdf, VID).reset_index()
start = origin(mdf, VID).reset_index()
end = destiny(mdf, VID).reset_index()

geoplotlib.dot(points_df, color="red", f_tooltip=tool_tip)
geoplotlib.dot(start, color="green", point_size=5, f_tooltip=tool_tip)
geoplotlib.dot(end, color="black", point_size=5, f_tooltip=tool_tip)
geoplotlib.show()

## b) Plot trajectory graph of vehicle

- Select vehicle id (vid): 4858_52
- build vehicle trajectory (dest_lat and dest_lon fields)
- use geoplotlib.graph function to plot the data

In [73]:
import geoplotlib

VID='6311_1'
graph_df = filter_by_id(mdf, VID).reset_index()
start = origin(mdf, VID).reset_index()
end = destiny(mdf, VID).reset_index()

graph_df.loc[:, 'dest_lat'] = graph_df.shift(-1).loc[:, 'lat']
graph_df.loc[:, 'dest_lon'] = graph_df.shift(-1).loc[:, 'lon']
graph_df = graph_df[:-1]

# Edge colors are based on distance between points
geoplotlib.graph(graph_df, src_lat='lat', src_lon='lon', dest_lat='dest_lat', dest_lon='dest_lon', linewidth=80, alpha=255)

#Plot (start, end) points
geoplotlib.dot(start, color="green", point_size=8, f_tooltip=tool_tip)
geoplotlib.dot(end, color="black", point_size=8, f_tooltip=tool_tip)
geoplotlib.show()



IndexError: positional indexers are out-of-bounds

## c) Animated trajectory graph of vehicle

- Select vehicle id (vid): 4858_52
- build animated vehicle trajectory (dest_lat and dest_lon fields)
- use new layer class to plot the points on each new frame

In [4]:
# Custom layer
from geoplotlib.layers import BaseLayer
from geoplotlib.layers import HotspotManager
from geoplotlib.utils import BoundingBox
from geoplotlib.core import BatchPainter
import random
import time

class DotDensityLayer(BaseLayer):

    def __init__(self, data, color=None, point_size=2, f_tooltip=None):
        """Create a dot density map
        :param data: data access object
        :param color: color
        :param point_size: point size
        :param f_tooltip: function to return a tooltip string for a point
        """
        self.frame_counter = 0
        self.data = data
        self.color = color
        if self.color is None:
            self.color = [255,0,0]
        self.point_size = point_size
        self.f_tooltip = f_tooltip

        self.hotspots = HotspotManager()


    def invalidate(self, proj):
        self.x, self.y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])



    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])
        if self.f_tooltip:
            for i in range(0, len(x)):
                record = {k: self.data[k][i] for k in self.data.keys()}
                self.hotspots.add_rect(x[i] - self.point_size, y[i] - self.point_size,
                                       2*self.point_size, 2*self.point_size,
                                       self.f_tooltip(record))

        self.painter.set_color(self.color)
        self.painter.points(x[self.frame_counter], y[self.frame_counter], 2*self.point_size, False)
        
        self.painter.batch_draw()
        picked = self.hotspots.pick(mouse_x, mouse_y)
        if picked:
            ui_manager.tooltip(picked)
        self.frame_counter += 1
        time.sleep(0.4)
        if self.frame_counter == len(x):
            self.frame_counter = 0


    def bbox(self):
        return BoundingBox.from_points(lons=self.data['lon'], lats=self.data['lat'])
        
geoplotlib.add_layer(DotDensityLayer(graph_df))
geoplotlib.show()

## d) Plot dots animation of all lines

- use new layer class to plot the points on each new frame

In [20]:
# Custom layer
from geoplotlib.layers import BaseLayer
from geoplotlib.layers import HotspotManager
from geoplotlib.utils import BoundingBox
from geoplotlib.core import BatchPainter
import random
import time

trajectories = mdf

class DotDensityLayer(BaseLayer):

    def __init__(self, data, color=None, point_size=2, f_tooltip=None):
        """Create a dot density map
        :param data: data access object
        :param color: color
        :param point_size: point size
        :param f_tooltip: function to return a tooltip string for a point
        """
        self.frame_counter = 23
        self.data = data
        self.last = self.data.drop_duplicates(subset=['vid'], keep='first')
        self.color = color
        if self.color is None:
            self.color = [255,0,0]
        self.point_size = point_size
        self.f_tooltip = f_tooltip

        self.hotspots = HotspotManager()


    def invalidate(self, proj):
        self.last = self.data.loc[self.data['time'] == self.frame_counter]
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        self.x, self.y = proj.lonlat_to_screen(lon_at, lat_at)
        


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        current = self.data.loc[self.data['time'] == self.frame_counter]
        
        self.last = pd.concat([self.last, current]).drop_duplicates(subset='vid', keep='last')
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        x, y = proj.lonlat_to_screen(lon_at, lat_at)
        
        if self.f_tooltip:
            for i in range(0, len(x)):
                record = {k: self.data[k][i] for k in self.data.keys()}
                self.hotspots.add_rect(x[i] - self.point_size, y[i] - self.point_size,
                                       2*self.point_size, 2*self.point_size,
                                       self.f_tooltip(record))

        self.painter.set_color(self.color)
        self.painter.points(x, y, 2*self.point_size, False)
        
        self.painter.batch_draw()
        picked = self.hotspots.pick(mouse_x, mouse_y)
        if picked:
            ui_manager.tooltip(picked)
        self.frame_counter += 1
        #time.sleep(0.4)


    def bbox(self):
        return BoundingBox.from_points(lons=self.data['lon'], lats=self.data['lat'])
        
geoplotlib.add_layer(DotDensityLayer(trajectories))
geoplotlib.show()

## e) Plot graph of all lines

- build vehicle trajectories of all lines (dest_lat and dest_lon fields)
- use geoplotlib.graph function to plot the data

In [16]:
# input_size_to_human = "small_"
if INPUT_SIZE in range(1, SMALL + 1):
    input_size_to_human = "small"
elif INPUT_SIZE in range(SMALL, MEDIUM + 1):
    input_size_to_human = "medium"
elif INPUT_SIZE == None or INPUT_SIZE > MEDIUM:
    input_size_to_human = "large"
else:
    raise Exception("Invalid input size")
                    
output_file =  ("../transformations/adds_dest_coordinates_%s.csv" % input_size_to_human)

def calculate_trajectories(force=False):
    import os.path
    if(force==False and os.path.exists(output_file)):
        print("Trajectories calculated in %s, use force=True to recalculate" % output_file)
        return

    counts = mdf.groupby(['vid'], as_index=False)

    print("Input Size: %s" % input_size_to_human)
    print("Number of unique vehicles: %s" % len(counts.count()))

    def shift_lat_lon(df):
        df.loc[:, 'dest_lat'] = df.shift(-1).loc[:, 'lat']
        df.loc[:, 'dest_lon'] = df.shift(-1).loc[:, 'lon']
        df.loc[:, 'start']
        return df

    res = counts.apply(shift_lat_lon)
    res.sort_values(['vid','time'], inplace=True)
    res.reset_index(drop=True, inplace=True)

    print("Writing to csv")
    res.to_csv(output_file, index=False)
    print("Written to %s" % output_file)
    
calculate_trajectories()


Trajectories calculated in ../transformations/adds_dest_coordinates_small.csv, use force=True to recalculate


In [16]:
datapoints = pd.read_csv("adds_dest_coordinates_small.csv", delimiter=",")
filtered = datapoints[datapoints['dest_lat'].notnull()]

geoplotlib.graph(data=filtered,
                 src_lat='lat', 
                 src_lon='lon', 
                 dest_lat='dest_lat',
                 dest_lon='dest_lon', 
                 linewidth=20,
                 color=[0,155,255],
                 alpha=16)

geoplotlib.show()

In [104]:
import pandas as pd
import geoplotlib
coordinates = '/home/tallys/git/master-research/src/transformations/adds_dest_coordinates_small.csv'
datapoints = pd.read_csv(coordinates, delimiter=",")
#filtered = datapoints[datapoints['dest_lat'].notnull()]

# geoplotlib.kde(datapoints, bw=5, cmap='Oranges', cmap_levels=10, show_colorbar=True, alpha=250)

# geoplotlib.show()
groups = datapoints.groupby(['vid'])#.apply(lambda x: x.first)
# a.apply(lambda x: x.first)
mydata = groups.first()
mydata["dest_lat"] = groups.last()["lat"]
mydata["dest_lon"] = groups.last()["lon"]


geoplotlib.dot(groups.first().reset_index(), color="red", f_tooltip=tool_tip)
geoplotlib.dot(groups.last().reset_index(), color="blue", f_tooltip=tool_tip)
                    
geoplotlib.graph(data=mydata,
                 src_lat='lat',
                 src_lon='lon', 
                 dest_lat='dest_lat',
                 dest_lon='dest_lon', 
                 linewidth=20,
#                  color=[0,155,255],
                 alpha=16)
geoplotlib.dot(groups.first().reset_index(), color="red", f_tooltip=tool_tip)
geoplotlib.dot(groups.last().reset_index(), color="blue", f_tooltip=tool_tip)

geoplotlib.show()
